In [ ]:
import nest_asyncio
nest_asyncio.apply()

import aiohttp
import asyncio
import json
import pandas as pd

# Define global variables for dataframes
student_df = None
engagement_df = None
ums_att_df = None
ums_avg_att_df = None
processed_attendance_data = pd.DataFrame()  # DataFrame to store processed attendance data
attendance_summary_data = pd.DataFrame()  # DataFrame to store attendance summary data

# Edoofy app information
edoofy_base_url = "https://edoofa-portal.bubbleapps.io/api/1.1/obj"
edoofy_bearer_token = "2cde31d8f48919a2db1467cc06a56132"
edoofy_headers = {'Authorization': f'Bearer {edoofy_bearer_token}'}

# UMS app information
ums_base_url = "https://app.edoofa.com/api/1.1/obj"
ums_bearer_token = "786720e8eb68de7054d1149b56cc04f9"
ums_headers = {'Authorization': f'Bearer {ums_bearer_token}'}

async def fetch_table_data(session, base_url, headers, table, constraints=None):
    records = []
    cursor = 0
    total_fetched = 0

    while True:
        params = {'limit': 100, 'cursor': cursor}
        if constraints:
            params['constraints'] = json.dumps(constraints)

        api_url = f"{base_url}/{table}"
        print(f"Fetching {table} data from {base_url}... Cursor: {cursor}")

        async with session.get(api_url, headers=headers, params=params) as response:
            if response.status != 200:
                print(f"Failed to fetch data from {table}: {await response.text()}")
                break

            data = await response.json()
            new_records = data['response']['results']
            records.extend(new_records)
            total_fetched += len(new_records)

            print(f"Fetched {len(new_records)} new records, Total fetched: {total_fetched}")

            cursor += 100

            if len(new_records) < 100:
                print(f"Exiting loop, fetched less than 100 records.")
                break

    df = pd.DataFrame(records)
    print(f"Fetched {len(df)} records for {table}.")
    return df

def map_students_to_engagement(student_df, engagement_df):
    if '_id' in student_df.columns and 'EWYL-group-name' in student_df.columns:
        student_id_to_EWYL = dict(zip(student_df['_id'], student_df['EWYL-group-name']))
        engagement_df['ewyl'] = engagement_df['student'].map(student_id_to_EWYL)
    else:
        print("Error: '_id' or 'EWYL-group-name' not in student_df columns")
    return engagement_df

def process_engagement_data(ums_att_df, engagement_df, student_df, ewyl_group):
    new_ums_df = pd.DataFrame(columns=['admissions-group-name', 'attendance-type', 'date',
                                       'ewyl-group-name', 'present', 'webinar'])
    student_to_kam_mapping = dict(zip(student_df['_id'], student_df['KAM-group-name']))
    student_to_ewyl_mapping = dict(zip(student_df['_id'], student_df['EWYL-group-name']))

    engagement_df['KAM-group-name'] = engagement_df['student'].map(student_to_kam_mapping)
    engagement_df['ewyl'] = engagement_df['student'].map(student_to_ewyl_mapping)

    student_latest_ums_date = pd.to_datetime(ums_att_df[ums_att_df['ewyl-group-name'] == ewyl_group]['date'].max())

    if pd.isnull(student_latest_ums_date):
        new_ums_df['ewyl-group-name'] = engagement_df['ewyl']
        new_ums_df['admissions-group-name'] = engagement_df['KAM-group-name']
        new_ums_df['attendance-type'] = engagement_df['engagement-type']
        new_ums_df['date'] = engagement_df['engagement-date']
        new_ums_df['present'] = engagement_df['daily-attendance']
        print("Adding all records")
    else:
        filtered_engagement_df = engagement_df[pd.to_datetime(engagement_df['engagement-date']) > student_latest_ums_date]
        new_ums_df['ewyl-group-name'] = filtered_engagement_df['ewyl']
        new_ums_df['admissions-group-name'] = filtered_engagement_df['KAM-group-name']
        new_ums_df['attendance-type'] = filtered_engagement_df['engagement-type']
        new_ums_df['date'] = filtered_engagement_df['engagement-date']
        new_ums_df['present'] = filtered_engagement_df['daily-attendance']
        print("Adding new records")

    return new_ums_df

def get_percentage(number_of_present, total_sessions):
    if total_sessions == 0:
        return 0
    return (number_of_present / total_sessions) * 100

async def process_att_summary(combined_df, ums_avg_att_df):
    global attendance_summary_data
    combined_df['date'] = pd.to_datetime(combined_df['date'])
    unique_students = combined_df['admissions-group-name'].unique()

    for student in unique_students:
        student_df = combined_df[combined_df['admissions-group-name'] == student]
        first_date = pd.to_datetime(student_df['date'].min())
        first_day_current_month = first_date.replace(day=1)

        while True:
            this_month_df = student_df[(student_df['date'].dt.year == first_day_current_month.year) &
                                       (student_df['date'].dt.month == first_day_current_month.month)]
            prev_month_df = student_df[student_df['date'] < first_day_current_month]

            total_sessions_this_month = len(this_month_df)
            attended_sessions_this_month = this_month_df['present'].sum()
            attendance_percentage_this_month = get_percentage(attended_sessions_this_month, total_sessions_this_month)

            if prev_month_df.empty:
                avg_att_till_last_month = 0
            else:
                total_sessions_till_last_month = len(prev_month_df)
                attended_sessions_till_last_month = prev_month_df['present'].sum()
                avg_att_till_last_month = get_percentage(attended_sessions_till_last_month, total_sessions_till_last_month)

            new_row = {
                'admissions-group-name': student,
                'attendance-percentage': attendance_percentage_this_month,
                'avg-att-percent-till-last-month': avg_att_till_last_month,
                'ewyl-group-name': student_df['ewyl-group-name'].iloc[0] if not student_df['ewyl-group-name'].empty else 'N/A',
                'first-day-of-month': first_day_current_month.strftime("%Y-%m-%d"),
                'month': first_day_current_month.strftime("%B"),
                'year': first_day_current_month.year
            }
            attendance_summary_data = pd.concat([attendance_summary_data, pd.DataFrame([new_row])], ignore_index=True)

            next_month_first_day = (first_day_current_month + pd.offsets.MonthBegin()).normalize()
            first_day_current_month = next_month_first_day

            if student_df[student_df['date'] >= first_day_current_month].empty:
                break

async def main():
    global student_df, engagement_this_month_df, engagement_previous_months_df, ums_att_df, ums_avg_att_df, processed_attendance_data
    
    async with aiohttp.ClientSession() as session:
        ums_att_df = await fetch_table_data(session, ums_base_url, ums_headers, "Attendance")
        ums_avg_att_df = await fetch_table_data(session, ums_base_url, ums_headers, "Attendance-Summary")
        
        student_constraints = [{'key': 'indian-edoofian', 'constraint_type': 'equals', 'value': 'yes'}]
        student_df = await fetch_table_data(session, edoofy_base_url, edoofy_headers, "Student", constraints=student_constraints)
        
        engagement_df = pd.DataFrame()
        for engagement_type in ['IE Call', 'IE Chat', 'Activity', 'Lesson']:
            engagement_data = await fetch_table_data(session, edoofy_base_url, edoofy_headers, "Engagement", [{'key': 'engagement-type', 'constraint_type': 'equals', 'value': engagement_type}])
            engagement_df = pd.concat([engagement_df, engagement_data], ignore_index=True)
        
        for _, student_row in student_df.iterrows():
            ewyl_group = student_row['EWYL-group-name']
            processed_df = process_engagement_data(ums_att_df, engagement_df, student_df, ewyl_group)
            
            if not processed_df.empty:
                processed_attendance_data = pd.concat([processed_attendance_data, processed_df], ignore_index=True)
            else:
                print(f"No data to process for student {ewyl_group}")

            await asyncio.sleep(1)  # Delay between processing each student

        if not processed_attendance_data.empty:
            processed_attendance_data.to_csv("processed_attendance_data.csv", index=False)
            print("Processed attendance data saved to 'processed_attendance_data.csv'.")

        if not processed_attendance_data.empty:
            await process_att_summary(processed_attendance_data, ums_avg_att_df)
            attendance_summary_data.to_csv("attendance_summary_data.csv", index=False)
            print("Attendance summary data saved to 'attendance_summary_data.csv'.")

await main()


Fetching Attendance data from https://app.edoofa.com/api/1.1/obj... Cursor: 0
Fetched 100 new records, Total fetched: 100
Fetching Attendance data from https://app.edoofa.com/api/1.1/obj... Cursor: 100
Fetched 100 new records, Total fetched: 200
Fetching Attendance data from https://app.edoofa.com/api/1.1/obj... Cursor: 200
Fetched 100 new records, Total fetched: 300
Fetching Attendance data from https://app.edoofa.com/api/1.1/obj... Cursor: 300
Fetched 100 new records, Total fetched: 400
Fetching Attendance data from https://app.edoofa.com/api/1.1/obj... Cursor: 400
Fetched 100 new records, Total fetched: 500
Fetching Attendance data from https://app.edoofa.com/api/1.1/obj... Cursor: 500
Fetched 100 new records, Total fetched: 600
Fetching Attendance data from https://app.edoofa.com/api/1.1/obj... Cursor: 600
Fetched 100 new records, Total fetched: 700
Fetching Attendance data from https://app.edoofa.com/api/1.1/obj... Cursor: 700
Fetched 100 new records, Total fetched: 800
Fetching A

Fetched 100 new records, Total fetched: 2200
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 2200
Fetched 100 new records, Total fetched: 2300
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 2300
Fetched 100 new records, Total fetched: 2400
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 2400
Fetched 100 new records, Total fetched: 2500
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 2500
Fetched 100 new records, Total fetched: 2600
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 2600
Fetched 100 new records, Total fetched: 2700
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 2700
Fetched 100 new records, Total fetched: 2800
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 2800
Fetched 17 new records, Tot

Fetched 100 new records, Total fetched: 3100
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 3100
Fetched 100 new records, Total fetched: 3200
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 3200
Fetched 100 new records, Total fetched: 3300
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 3300
Fetched 100 new records, Total fetched: 3400
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 3400
Fetched 100 new records, Total fetched: 3500
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 3500
Fetched 100 new records, Total fetched: 3600
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 3600
Fetched 100 new records, Total fetched: 3700
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 3700
Fetched 100 new records, To

Fetched 100 new records, Total fetched: 4900
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 4900
Fetched 100 new records, Total fetched: 5000
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 5000
Fetched 100 new records, Total fetched: 5100
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 5100
Fetched 100 new records, Total fetched: 5200
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 5200
Fetched 100 new records, Total fetched: 5300
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 5300
Fetched 100 new records, Total fetched: 5400
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 5400
Fetched 100 new records, Total fetched: 5500
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 5500
Fetched 100 new records, To

Adding new records
Adding new records
Adding new records
Adding all records
Adding new records
Adding all records
